# Torpe Blockchain (तोर्पे ब्लोक्कचैन)
### A minimal blockchain data structure to understand the blockchain basics like hashing, nonce, states, genesis blocks etc.

![title](blockchain.jpg)

### Hash Function

In [1]:
import hashlib
def hash_sha256(raw):
    raw = str(raw).encode('utf-8')
    return hashlib.sha256(raw).hexdigest() 

In [2]:
hash_sha256('torpe_blockchain')

'e5367197a1f12480ec761306f2fa9d15b494d5a80e5a806713df9e60943f4faf'

In [3]:
hash_sha256('torpe_blockchain')

'e5367197a1f12480ec761306f2fa9d15b494d5a80e5a806713df9e60943f4faf'

In [4]:
hash_sha256('monkey_brain')

'35ba7e0c2afd3e1175399e99c7e307ea78e62c9dd6e3cb39afc9604043a55e4b'

SHA-256 collisions have not been found yet. 

### Nonce

> Number that can only be used once

An arbitrary numer used in cryptography to ensure uniqueness and prevent the rerunning of transactions (known as replay attack). 

In [5]:
def hash_sha256_nonce(raw):
    raw_bytes = str(raw).encode('utf-8')
    hashed = hashlib.sha256(raw_bytes).hexdigest() 
    nonce = 0
    while (hashed[:5] != '00000'):
        nonce = nonce+1
        raw.update({'nonce': nonce})
        raw_bytes = str(raw).encode('utf-8')
        hashed = hashlib.sha256(raw_bytes).hexdigest() 
    return raw, hashed

In [6]:
print (hash_sha256_nonce({'nonce': 0}))

({'nonce': 396548}, '0000063b8bbf3f9ebfecf2b47c1f0e0aedf0d74a8480d47e6c1dab5e647ea991')


## Block

Blocks hold batches of valid transactions that are hashed and encoded into a Merkle tree Each block includes the cryptographic hash of the prior block in the blockchain, linking the two. The linked blocks form a chain. This iterative process confirms the integrity of the previous block, all the way back to the original genesis block.

### Genesis Block

A genesis block or block0 is the first block of a block chain. The genesis block is almost always hardcoded into the software of the applications that utilize its block chain. It is a special case in that it does not reference a previous block

In [7]:
import datetime

In [8]:
# Lets assume 5 person were given 100 coins each
state = {
    'Person_1': 100, 
    'Person_2': 100, 
    'Person_3': 100, 
    'Person_4': 100, 
    'Person_5': 100
}

In [9]:
block0_data = {
    'timestamp': datetime.datetime.now(),
    'index': 0,
    'previous': None,
    'transactions': [state], 
    'nonce': 0
}

raw, hashed = hash_sha256_nonce(block0_data)
block0 = {
    'hash': hashed,
    'data': raw,
}

In [10]:
block0

{'data': {'index': 0,
  'nonce': 208993,
  'previous': None,
  'timestamp': datetime.datetime(2019, 1, 24, 23, 42, 18, 132177),
  'transactions': [{'Person_1': 100,
    'Person_2': 100,
    'Person_3': 100,
    'Person_4': 100,
    'Person_5': 100}]},
 'hash': '00000d0bafe93f33ae8bf8ec4e2d5033a64529bcb0ead9c9226ba2180329d88e'}

This is the genesis block or block 0 here.

### Transactions

In [11]:
import random
def random_transaction(state):
    temp_list = list(state.keys())
    random.shuffle(temp_list)
    # randomly select two persons
    first_person = temp_list.pop()
    second_person = temp_list.pop()
    receive = random.randint(1, 10)
    give = -receive
    return {
        first_person:receive, 
        second_person:give
    }

In [12]:
test_transactions = [random_transaction(state) for x in range(5)]

In [13]:
test_transactions

[{'Person_2': -8, 'Person_3': 8},
 {'Person_2': -9, 'Person_5': 9},
 {'Person_2': -10, 'Person_5': 10},
 {'Person_2': -8, 'Person_3': 8},
 {'Person_2': -2, 'Person_5': 2}]

### Updating State

In [14]:
def update_state(transaction, state):
    state = state.copy()
    for key in transaction:
        state[key] = state.get(key, 0) + transaction[key]
    return state

In [15]:
for transaction in test_transactions:
    state = update_state(transaction, state)

In [16]:
state

{'Person_1': 100,
 'Person_2': 63,
 'Person_3': 116,
 'Person_4': 100,
 'Person_5': 121}

### Valid Transactions

In [17]:
def check_transaction_validity(transaction, state):
    # check neg vs pos
    if sum(transaction.values()) is not 0:
        return False
    # check if amount in wallet to give
    for key in transaction.keys():
        if state.get(key, 0) + transaction[key] < 0:
            return False
    return True

In [18]:
for transaction in test_transactions:
    print (check_transaction_validity(transaction, state))

True
True
True
True
True


In [19]:
# No balance
print (check_transaction_validity({'A': 5, 'B': -5}, {'A': 0, 'B': 0}))

False


In [20]:
# Bad transaction
print (check_transaction_validity({'A': 5, 'B': 5}, {'A': 50, 'B': 50}))

False


### Initial State

In [21]:
# Let us reset
# Lets assume 5 person were given 100 coins each
state = {
    'Person_1': 100, 
    'Person_2': 100, 
    'Person_3': 100, 
    'Person_4': 100, 
    'Person_5': 100
}

blockchain = []

In [22]:
# Adding the genesis block
blockchain.append(block0)

In [23]:
blockchain

[{'data': {'index': 0,
   'nonce': 208993,
   'previous': None,
   'timestamp': datetime.datetime(2019, 1, 24, 23, 42, 18, 132177),
   'transactions': [{'Person_1': 100,
     'Person_2': 100,
     'Person_3': 100,
     'Person_4': 100,
     'Person_5': 100}]},
  'hash': '00000d0bafe93f33ae8bf8ec4e2d5033a64529bcb0ead9c9226ba2180329d88e'}]

### Non-genesis block / New block

In [24]:
def new_block(transactions, blockchain):
    previous_block = blockchain[-1]
    data = {
        'timestamp': datetime.datetime.now(),
        'index': previous_block['data']['index'] + 1,
        'previous': previous_block['hash'],
        'transactions': transactions,
        'nonce': 0
        }
    
    raw, hashed = hash_sha256_nonce(data)
    block = {'hash': hashed, 'data': raw}
    return block

In [25]:
sample_transactions = [random_transaction(state) for x in range(50)]

In [26]:
sample_transactions

[{'Person_3': -3, 'Person_5': 3},
 {'Person_2': 4, 'Person_5': -4},
 {'Person_1': -7, 'Person_5': 7},
 {'Person_1': 7, 'Person_2': -7},
 {'Person_1': -10, 'Person_4': 10},
 {'Person_2': -8, 'Person_3': 8},
 {'Person_1': -3, 'Person_2': 3},
 {'Person_4': -4, 'Person_5': 4},
 {'Person_1': 1, 'Person_5': -1},
 {'Person_2': 1, 'Person_5': -1},
 {'Person_2': -9, 'Person_5': 9},
 {'Person_2': -1, 'Person_3': 1},
 {'Person_2': 4, 'Person_3': -4},
 {'Person_1': 7, 'Person_4': -7},
 {'Person_4': 10, 'Person_5': -10},
 {'Person_1': 6, 'Person_4': -6},
 {'Person_1': 7, 'Person_4': -7},
 {'Person_1': 2, 'Person_3': -2},
 {'Person_3': -7, 'Person_4': 7},
 {'Person_2': -10, 'Person_4': 10},
 {'Person_4': -2, 'Person_5': 2},
 {'Person_2': 7, 'Person_5': -7},
 {'Person_1': -10, 'Person_2': 10},
 {'Person_1': 3, 'Person_5': -3},
 {'Person_3': -8, 'Person_4': 8},
 {'Person_1': -7, 'Person_4': 7},
 {'Person_3': -3, 'Person_4': 3},
 {'Person_2': 10, 'Person_4': -10},
 {'Person_1': -10, 'Person_5': 10},
 {

### Transactions per block

Bitcoin blocks used to contain fewer than 200 transactions and the largest number of transactions in a block was 1,976 at the time this answer was originally written (May 2013). In meanwhile (November 2017) the average number of transaction per block is well above 1500 with peaks above 2200.

In [27]:
# Although block size is 
transactions_per_block = 5
transaction_block = []

for transaction in sample_transactions:
    if check_transaction_validity(transaction, state):
        state = update_state(transaction, state)
        transaction_block.append(transaction)
        
        if len(transaction_block) >= transactions_per_block:
            blockchain.append(new_block(transaction_block, blockchain))
            transaction_block = []

In [28]:
import pprint
pp = pprint.PrettyPrinter()
for block in blockchain:
    pp.pprint(block)
    print('\n************************************************************************************\n')

{'data': {'index': 0,
          'nonce': 208993,
          'previous': None,
          'timestamp': datetime.datetime(2019, 1, 24, 23, 42, 18, 132177),
          'transactions': [{'Person_1': 100,
                            'Person_2': 100,
                            'Person_3': 100,
                            'Person_4': 100,
                            'Person_5': 100}]},
 'hash': '00000d0bafe93f33ae8bf8ec4e2d5033a64529bcb0ead9c9226ba2180329d88e'}

************************************************************************************

{'data': {'index': 1,
          'nonce': 4353465,
          'previous': '00000d0bafe93f33ae8bf8ec4e2d5033a64529bcb0ead9c9226ba2180329d88e',
          'timestamp': datetime.datetime(2019, 1, 24, 23, 42, 19, 660606),
          'transactions': [{'Person_3': -3, 'Person_5': 3},
                           {'Person_2': 4, 'Person_5': -4},
                           {'Person_1': -7, 'Person_5': 7},
                           {'Person_1': 7, 'Person_2': -7},
 

### The current state
#### Syncing for the first time

In [29]:
def validate_block(block, parent, state):    
    error_msg = 'Error in %d' % block['data']['index']

    # check block hash
    assert block['hash'] == hash_sha256(block['data']), error_msg

    # check block indices
    assert block['data']['index'] == parent['data']['index'] + 1, error_msg

    # check previous hash
    assert block['data']['previous'] == parent['hash'], error_msg
    
    # validate all transactions
    for transaction in block['data']['transactions']:
        assert check_transaction_validity(transaction, state), error_msg
        state = update_state(transaction, state)
        
    return state

In [30]:
def check_chain(blockchain):
    state = {}

    for transaction in blockchain[0]['data']['transactions']:
        state = update_state(transaction, state)

    parent = blockchain[0]
    
    for block in blockchain[1:]:
        state = validate_block(block, parent, state)
        parent = block

    return state

check_chain(blockchain)

{'Person_1': 75,
 'Person_2': 92,
 'Person_3': 106,
 'Person_4': 142,
 'Person_5': 85}